In [4]:
#libraries to be used later ; #using python 3.8.10 env 
import os
import numpy as np
from glob import glob
# import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import randint
import subprocess
import nibabel as nib

In [ ]:
def main():
    #inputs: Gad T1 Images (from greedy directory) and Nongad T1 images (from fmriprep folders), creating corresponding subject directory
    gad_rigid_t1= glob('/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/passing_dataset/rigid/*/*_acq-gad_resampled_T1w.nii.gz') # gad images who's corresponding nongad images underwent a rigid transform
    gad_affine_t1= glob('/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/passing_dataset/affine/*/*_acq-gad_resampled_T1w.nii.gz')# gad images who's corresponding nongad images underwent an affine transform
    gad_t1 = gad_rigid_t1 + gad_affine_t1#total list of QC'ed gad image paths
    gad_t1.sort(key=lambda x: os.path.basename(x).split('_')[0][0:])#sorting gad image paths alphabetically by subject name

    nongad_rigid_t1= glob('/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/passing_dataset/rigid/*/*acq-nongad_desc-rigid_resliced_T1w.nii.gz') # nongad images which underwent a rigid transform
    nongad_affine_t1 = glob('/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/passing_dataset/affine/*/*acq-nongad_desc-affine_resliced_T1w.nii.gz') #nongad images which underwent an affine transform
    nongad_t1= nongad_rigid_t1 + nongad_affine_t1#total list of QC'ed nongad image paths
    nongad_t1.sort(key=lambda x: os.path.basename(x).split('_')[0][0:])#sorting nongad image paths alphabetically by subject name

    synthstrip_path = "/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/synthstrip" #where skull stripped outputs will go
    path_c3d='/home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/c3d' #convert 3d software path

    list_sub=[]
    for path in gad_t1:##looping through list to add subject names to subject list
    sub = os.path.basename(path.split("_")[1])#retrieving subject number "ie. sub-P001"
    list_sub.append(sub)
    if not os.path.exists(f'{synthstrip_path}/{sub}'):#if c3d subject path does not exist, create it
        !mkdir {synthstrip_path}/{sub}
    if not os.path.exists(f'{path_c3d}/{sub}'):#if c3d subject path does not exist, create it
        !mkdir {path_c3d}/{sub}

    for i,path in enumerate(gad_t1):
        strip_arg = f'-i {path} -o /home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/synthstrip/{list_sub[i]}/{list_sub[i]}_acq-gad_stripped_b3_T1w.nii.gz -m /home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/synthstrip/{list_sub[i]}/{list_sub[i]}_acq-gad_mask_b3_T1w.nii -b 3'
        synthstrip(strip_arg)
        strip_arg = f'-i {path} -o /home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/synthstrip/{list_sub[i]}/{list_sub[i]}_acq-gad_stripped_b4_T1w.nii.gz -m /home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/synthstrip/{list_sub[i]}/{list_sub[i]}_acq-gad_mask_b4_T1w.nii -b 4'
        synthstrip(strip_arg)

    #c3d patching metrics
    radius_vector = np.array([31,31,31])
    radius_arg = 'x'.join([str(rad) for rad in radius_vector])
    frequency = 150 #frequency of voxels to be sampled
    num_rand= 5  #Number of randomly rotated patches
    sigma_angle = 30 # standard dev of rotation angle that is normally distrubuted
  
    for i,sub in enumerate(list_sub):
        #applying c3d using b=3 masks, after qc can switch out patches for b=4
        c3d(f'{nongad_t1[i]} {gad_t1[i]} /home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/synthstrip/{sub}/{sub}_acq-gad_mask_b3_T1w.nii -xpa {num_rand} {sigma_angle} -xp /home/ROBARTS/fogunsanya/graham/scratch/degad/derivatives/c3d/{sub}/{sub}_degad_samples.dat {radius_arg} {frequency}')
        print(f"Done {sub}")
main()

In [1]:
def synthstrip(args):
    """
        Function that uses synthstrip command line tool to remove skull from brain, by taking input argument as a single string
        Inputs:
            args String: argument that will be parsed by synthstrip command
        Output:
            None, produces skull stripped outputs in given directory
        
        To get usage:
            singularity run /home/ROBARTS/fogunsanya/graham/scratch/containers/synthstrip.1.3.sif -h
    """
    list_args = args.split(' ')
    synthstrip_path= "/home/ROBARTS/fogunsanya/graham/scratch/containers/synthstrip.1.3.sif"
    cmd = f'singularity run {synthstrip_path}'
    cmd_list = cmd.split(' ') + list_args
    try:
        output = subprocess.check_output(cmd_list,
                                            stderr=subprocess.STDOUT) 
    
    except subprocess.CalledProcessError as e:
        print(e.output.decode('utf-8'))
    else:
        print(output.decode("utf-8"))
    

In [ ]:
#defining a function that can help with using c3d which is a command line based tool (accessible through CBS)
def c3d(args):
  """ 
  Runs c3d, taking arguments as a single string. 

  Inputs:
    args String: arguments to c3d command as a single string


  Example:

  To get usage info:
  > c3d('-h') 

  To get usage for a specific command:
  > c3d('-h extract-patches')

  """
  list_args = args.split(' ')
  cmd_list = ['c3d'] + list_args
  try:
    output = subprocess.check_output(cmd_list,
                                    stderr=subprocess.STDOUT) 
  except subprocess.CalledProcessError as e:
    print(e.output.decode('utf-8'))
  else:
    print(output.decode("utf-8"))
   